# Data Update

Use to catch up data. 

Last use: Jan 5.

In [1]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta

# Today's release

In [2]:
def get_data():
    '''
    Pulls and cleans daily csv.
    '''
    
    # downloading CSV
    url = 'https://data.ontario.ca/dataset/f4112442-bdc8-45d2-be3c-12efae72fb27/resource/455fd63b-603d-4608-8216-7d8647f43350/download/conposcovidloc.csv'
    req = requests.get(url)
    url_content = req.content
    
    # saving unedited file as backup
    filename = './data/ontario_raw.csv'
    csv_file = open(filename, 'wb')
    csv_file.write(url_content)
    csv_file.close()
    print(f'Downloaded data; stored raw at {filename}')
    
    # cleaning file
    df = pd.read_csv('./data/ontario_raw.csv')
    df = df.drop(['Row_ID', 'Reporting_PHU_City',
                'Reporting_PHU_Address',
                'Reporting_PHU_Postal_Code', 
                'Reporting_PHU_Website',
                'Reporting_PHU_Latitude',
                'Reporting_PHU_Longitude',
                'Outbreak_Related',
                'Accurate_Episode_Date',
                'Test_Reported_Date',
                'Case_AcquisitionInfo',
                'Client_Gender',
                'Specimen_Date',
                'Outcome1'], axis=1)
    df = df.rename(columns={"Case_Reported_Date": "reported",
                            "Age_Group": "age",
                            'Reporting_PHU': 'unit'})
    df = df.set_index(['unit'])
    df = df.sort_values(by=['unit','reported'],
                        ascending=[True,False])
    
    # saving cleaned file
    filename = './data/ontario_data.csv'
    df.to_csv(filename)
    print(f'Stored clean data at {filename}.')
    
    # renaming units to match StatsCan convention
    file = pd.read_csv('./data/unit_names.csv')
    names = {x:y for x,y in zip(file['original'],file['rename'])}
    df = df.rename(index=names)
    
    return df

In [3]:
df = get_data()

Downloaded data; stored raw at ./data/ontario_raw.csv
Stored clean data at ./data/ontario_data.csv.


# Total cases

In [39]:
dates = pd.date_range(start='2020-10-27', end='2021-01-05')

In [40]:
# total counts on sliced data
units_canada = ['Middlesex-London Health Unit','Durham Regional Health Unit',
         'Halton Regional Health Unit','City of Hamilton Health Unit',
        'City of Ottawa Health Unit','Waterloo Health Unit',
        'Windsor-Essex County Health Unit']
ind = ['London','Durham','Halton','Hamilton',
           'Ottawa','Waterloo','Windsor']

In [41]:
dfs = []
# slicing sheet by date
for i in range(len(dates)):
    date = dates[i]
    included_dates = [str(d)[:10] for d in
                      pd.date_range(start='2020-2-25', end=date)]
    data = df[df['reported'].isin(included_dates)]
    
    # calculating 
    total_counts = []
    for unit in units_canada:
        d = {}
        unit_grp = data.groupby('unit').get_group(unit)
        d['total'] = len(unit_grp['age'])
        total_counts.append(d)
        
    # assembling into DF, appending to DF list
    total_df = pd.DataFrame(total_counts, index=units_canada)
    dfs.append(total_df)

In [42]:
new_totals = pd.concat(dfs, keys=dates)
new_totals = new_totals.unstack()[::-1]
new_totals.rename(columns={x:y for x,y in zip(units_canada,ind)},
                 inplace=True)
new_totals.index.name = 'date'
new_totals.columns = new_totals.columns.droplevel()
new_totals

,London,Durham,Halton,Hamilton,Ottawa,Waterloo,Windsor
date,,,,,,,
2021-01-05,3824,8155,6278,6870,10692,6594,8736
2021-01-04,3787,8062,6253,6828,10637,6508,8689
2021-01-03,3631,7931,6138,6708,10530,6377,8557
2021-01-02,3507,7786,6019,6558,10433,6240,8405
2021-01-01,3438,7604,5951,6443,10340,6142,8271
...,...,...,...,...,...,...,...
2020-10-31,1139,3039,2083,1852,7129,2168,2844
2020-10-30,1131,3016,2056,1817,7047,2146,2831
2020-10-29,1126,2984,2033,1782,6926,2127,2823


In [44]:
old_totals = pd.read_csv('./data/output/ontario_total.csv').set_index('date')
old_totals

,London,Durham,Halton,Hamilton,Ottawa,Waterloo,Windsor
date,,,,,,,
2021-01-06,3824.0,8155.0,6278.0,6870.0,10692.0,6594.0,8736.0
2021-01-05,3824.0,8155.0,6278.0,6870.0,10692.0,6594.0,8736.0
2021-01-04,3787.0,8062.0,6253.0,6828.0,10637.0,6508.0,8689.0
2021-01-03,3631.0,7931.0,6138.0,6708.0,10530.0,6377.0,8557.0
2021-01-02,3507.0,7786.0,6019.0,6558.0,10433.0,6240.0,8405.0
...,...,...,...,...,...,...,...
2020-02-29,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2020-02-28,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2020-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
final_total = pd.concat([new_totals, old_totals])
final_total.index = pd.to_datetime(final_total.index)
final_total

,London,Durham,Halton,Hamilton,Ottawa,Waterloo,Windsor
date,,,,,,,
2021-01-05,3824.0,8155.0,6278.0,6870.0,10692.0,6594.0,8736.0
2021-01-04,3787.0,8062.0,6253.0,6828.0,10637.0,6508.0,8689.0
2021-01-03,3631.0,7931.0,6138.0,6708.0,10530.0,6377.0,8557.0
2021-01-02,3507.0,7786.0,6019.0,6558.0,10433.0,6240.0,8405.0
2021-01-01,3438.0,7604.0,5951.0,6443.0,10340.0,6142.0,8271.0
...,...,...,...,...,...,...,...
2020-02-29,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2020-02-28,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2020-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
ds = pd.date_range(start='2020-02-25',
                                 end='2021-01-06')
for x in final_total.index:
    if x not in ds:
        print(x)

In [47]:
final_total.to_csv('./data/output/ontario_total.csv')

# New, 7day and 14day

- using `final_total` from above

In [48]:
total = pd.read_csv('./data/output/ontario_total.csv').set_index('date')
total

,London,Durham,Halton,Hamilton,Ottawa,Waterloo,Windsor
date,,,,,,,
2021-01-05,3824.0,8155.0,6278.0,6870.0,10692.0,6594.0,8736.0
2021-01-04,3787.0,8062.0,6253.0,6828.0,10637.0,6508.0,8689.0
2021-01-03,3631.0,7931.0,6138.0,6708.0,10530.0,6377.0,8557.0
2021-01-02,3507.0,7786.0,6019.0,6558.0,10433.0,6240.0,8405.0
2021-01-01,3438.0,7604.0,5951.0,6443.0,10340.0,6142.0,8271.0
...,...,...,...,...,...,...,...
2020-02-29,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2020-02-28,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2020-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# getting new counts for each column into list
news = {}
for unit in total.columns:
    s = total[unit]
    new = s.diff(periods = -1)
    news[unit] = new

In [50]:
ndf = pd.DataFrame(news)

In [51]:
ndf

,London,Durham,Halton,Hamilton,Ottawa,Waterloo,Windsor
date,,,,,,,
2021-01-05,37.0,93.0,25.0,42.0,55.0,86.0,47.0
2021-01-04,156.0,131.0,115.0,120.0,107.0,131.0,132.0
2021-01-03,124.0,145.0,119.0,150.0,97.0,137.0,152.0
2021-01-02,69.0,182.0,68.0,115.0,93.0,98.0,134.0
2021-01-01,81.0,138.0,119.0,127.0,124.0,146.0,180.0
...,...,...,...,...,...,...,...
2020-02-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-28,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2020-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
# getting 7-day averages
avgs = {}
for unit in ndf.columns:
    s = ndf[unit][::-1]
    avgs[unit + '_7'] = s.rolling(window=7).mean()[::-1]

In [53]:
adf = pd.DataFrame(avgs)

In [54]:
adf

,London_7,Durham_7,Halton_7,Hamilton_7,Ottawa_7,Waterloo_7,Windsor_7
date,,,,,,,
2021-01-05,86.857143,139.000000,96.285714,122.571429,103.285714,108.142857,159.428571
2021-01-04,100.571429,146.000000,101.285714,137.857143,115.428571,113.571429,181.285714
2021-01-03,90.285714,144.571429,101.142857,140.142857,107.714286,105.428571,192.714286
2021-01-02,85.571429,134.000000,96.285714,137.857143,102.571429,93.142857,202.428571
2021-01-01,87.857143,115.714286,100.714286,134.428571,94.857143,87.000000,205.857143
...,...,...,...,...,...,...,...
2020-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
fdf = pd.concat([ndf, adf], axis=1)
fdf = fdf[['London', 'London_7',
              'Durham','Durham_7',
              'Halton','Halton_7',
              'Hamilton','Hamilton_7',
              'Ottawa','Ottawa_7',
              'Waterloo','Waterloo_7',
              'Windsor','Windsor_7']]
fdf.to_csv('./data/output/ontario_new.csv')

In [56]:
fdf

,London,London_7,Durham,Durham_7,Halton,Halton_7,Hamilton,Hamilton_7,Ottawa,Ottawa_7,Waterloo,Waterloo_7,Windsor,Windsor_7
date,,,,,,,,,,,,,,
2021-01-05,37.0,86.857143,93.0,139.000000,25.0,96.285714,42.0,122.571429,55.0,103.285714,86.0,108.142857,47.0,159.428571
2021-01-04,156.0,100.571429,131.0,146.000000,115.0,101.285714,120.0,137.857143,107.0,115.428571,131.0,113.571429,132.0,181.285714
2021-01-03,124.0,90.285714,145.0,144.571429,119.0,101.142857,150.0,140.142857,97.0,107.714286,137.0,105.428571,152.0,192.714286
2021-01-02,69.0,85.571429,182.0,134.000000,68.0,96.285714,115.0,137.857143,93.0,102.571429,98.0,93.142857,134.0,202.428571
2021-01-01,81.0,87.857143,138.0,115.714286,119.0,100.714286,127.0,134.428571,124.0,94.857143,146.0,87.000000,180.0,205.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
2020-02-28,0.0,NaN,2.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
2020-02-27,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN


# 14-day change

In [2]:
import pandas as pd

In [3]:
ndf = pd.read_csv('./data/output/ontario_new.csv').set_index('date')
ndf

,London,London_7,Durham,Durham_7,Halton,Halton_7,Hamilton,Hamilton_7,Ottawa,Ottawa_7,Waterloo,Waterloo_7,Windsor,Windsor_7
date,,,,,,,,,,,,,,
2020-10-13,43.0,10.571429,25.0,17.428571,49.0,28.142857,49.0,21.857143,235.0,90.857143,23.0,11.000000,22.0,6.142857
2020-10-12,0.0,6.714286,0.0,15.714286,0.0,26.285714,0.0,17.428571,0.0,77.571429,0.0,9.571429,0.0,3.428571
2020-10-11,0.0,8.142857,0.0,17.714286,0.0,29.142857,0.0,20.428571,0.0,89.571429,0.0,11.571429,0.0,5.428571
2020-10-10,4.0,9.142857,8.0,19.428571,25.0,32.428571,2.0,22.428571,29.0,100.714286,9.0,14.571429,3.0,6.142857
2020-10-09,3.0,9.571429,39.0,20.000000,41.0,31.285714,54.0,22.857143,103.0,106.000000,19.0,15.285714,8.0,6.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-01,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
2020-02-29,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
2020-02-28,0.0,0.000000,2.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [9]:
# 14 day loop
unit_names = ['London','Durham','Halton','Hamilton',
           'Ottawa','Waterloo','Windsor']

changes = {}
for unit in unit_names:
    s = ndf[unit][::-1]
    t = s[len(s)-14:]
    p, q = t[:7], t[7:]
    first = sum(q)
    second = sum(p)
    c = (first / second * 100) - 100
    changes[unit] = c

In [14]:
cdf = pd.DataFrame(changes, index=['2020-10-13'])
cdf.index.name = 'date'
cdf.to_csv('./data/output/ontario_change.csv')

In [128]:
# algorithm to calculate change
ldn = ndf.London[::-1]
t = ldn[25-14:25]
p, q = t[:7], t[7:]
print(p,q)
first = sum(q)
second = sum(p)
print(first, second)
print((first / second * 100) - 100)

date
2020-03-08    0.0
2020-03-09    0.0
2020-03-10    0.0
2020-03-11    0.0
2020-03-12    1.0
2020-03-13    0.0
2020-03-14    0.0
Name: London, dtype: float64 date
2020-03-15    0.0
2020-03-16    2.0
2020-03-17    1.0
2020-03-18    0.0
2020-03-19    4.0
2020-03-20    3.0
2020-03-21    0.0
Name: London, dtype: float64
10.0 1.0
900.0


# Demo

- using total DF
- using original `ontario_data` df

In [136]:
total.head()

,London,Durham,Halton,Hamilton,Ottawa,Waterloo,Windsor
date,,,,,,,
2020-10-12,957.0,2371.0,1467.0,1344.0,5419.0,1887.0,2707.0
2020-10-11,957.0,2371.0,1467.0,1344.0,5419.0,1887.0,2707.0
2020-10-10,957.0,2371.0,1467.0,1344.0,5419.0,1887.0,2707.0
2020-10-09,953.0,2363.0,1442.0,1342.0,5390.0,1878.0,2704.0
2020-10-08,950.0,2324.0,1401.0,1288.0,5287.0,1859.0,2696.0


In [141]:
org = pd.read_csv('./data/ontario_data.csv').set_index('unit')
org

,reported,age
unit,,
Algoma Public Health Unit,2020-10-09,30s
Algoma Public Health Unit,2020-10-09,20s
Algoma Public Health Unit,2020-10-06,60s
Algoma Public Health Unit,2020-10-06,30s
Algoma Public Health Unit,2020-10-05,40s
...,...,...
York Region Public Health Services,2020-02-29,60s
York Region Public Health Services,2020-02-29,70s
York Region Public Health Services,2020-02-28,40s
